In [3]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister
import logging
import random

In [4]:

logger = logging.getLogger(__name__)

def random_quantum_circuit(num_q:int, num_a:int, num_g:int=10) -> QuantumCircuit:
    in_q = QuantumRegister(num_q, name='cq')
    an_q = QuantumRegister(num_a, name='aq')
    
    circuit = QuantumCircuit(in_q, an_q)
    
    for i in range(num_g):
        
        if random.random() < 0.75: # Input acts on Ancilla    
            control_q = in_q
            target_q = an_q

        else: # Ancilla acts on input
            control_q = an_q
            target_q = in_q

        num_controls = random.randint(1, control_q.size)
        controls = random.sample(range(control_q.size), num_controls)  # Get control qubit/s
        target = random.randrange(target_q.size) # Get target qubit
        print(num_controls, controls, target)
        circuit.mcx([control_q[cq] for cq in controls],target_q[target]) 

    logger.info(f'Built circuit with {num_q} input, {num_a} ancilla and {num_g} gates.')
    return circuit

In [26]:
num_q=4
num_a=4
num_g=5
circuit = random_quantum_circuit(num_q,num_a,num_g)

1 [0] 1
1 [1] 3
2 [3, 0] 1
2 [1, 0] 2
3 [3, 1, 0] 0


In [27]:
def breakdown_qubit(qubit: qiskit.circuit.Qubit):
    return {'name':qubit._register.name, 'wire':qubit._index, 'qubit':qubit, 'label':qubit._register.name+str(qubit._index)}



In [28]:
ancillas_list = [breakdown_qubit(q)['label'] for q in circuit.qubits][-num_a:]

In [29]:
ancillas_list

['aq0', 'aq1', 'aq2', 'aq3']